In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import folium
import requests
import urllib.request
#!conda install python3-bs4 --yes
!conda install beautifulsoup4 --yes
!conda install lxml --yes
from bs4 import BeautifulSoup
!conda install -c conda-forge geopy --yes
from geopy import geocoders
print("libraries imported")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> pkgs/main::ca-certificates-2020.1.1-0
  certifi            conda-forge::certifi-2020.4.5.1-py36h~ --> pkgs/main::certifi-2020.4.5.1-py36_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_0 --> pkgs/main::openssl-1.1.1g-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The fol

In [2]:
wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(wiki, header=0)[0]

df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
uns_borough = df[df['Borough']!="Not assigned"]
uns_borough.head(50)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#count the number of unique postal codes

In [4]:
print(len(df['Postal Code'].unique()))

180


Here we can see that all the postal codes are unique, so the corresponding neighborhoods are assigned to them

In [45]:
#check for neighborhood if not assigned value is there
for notAssign in uns_borough['Neighborhood']:
    if notAssign == "Not assigned":
        print(uns_borough[notAssign])

As we have checked that no "Not assigned" value is there in neighborhood column just for logic now we will now fill the Not assigned neighborhood with the corresponding borough

In [42]:
for bor, neigh in zip(uns_borough['Borough'],uns_borough['Neighborhood']):
    if neigh=='Not assigned':
        uns_borough[neigh] = uns_borough[bor]
        
uns_borough.tail()        

,Postal Code,Borough,Neighborhood
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Submission 1: Shape of the DataFrame

In [43]:
print("Shape of the DataFrame is:",uns_borough.shape)

Shape of the DataFrame is: (103, 3)


In [14]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Get the shape of the excel sheet

In [16]:
df_geo.shape

(103, 3)

We will check if both the dataframes(uns_borough and df_geo) have same sequence of postal codes

In [30]:
#For that let's first sort the dataframes based on Postal Code column
df1 = uns_borough.sort_values('Postal Code')
df1.head()

,Postal Code,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


In [31]:
df2 = df_geo.sort_values('Postal Code')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now we will append the values of Latitude and Longitude in First dataframe

In [33]:
for pc1, pc2 in zip(df1['Postal Code'], df2['Postal Code']):
    if pc1 == pc2:
        df1['Latitude'] = df2['Latitude']
        df1['Longitude'] = df2['Longitude']
df1.head()        

,Postal Code,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.692657,-79.264848
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.778517,-79.346556
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.725900,-79.340923
36,M1G,Scarborough,Woburn,43.695344,-79.318389
45,M1H,Scarborough,Cedarbrae,43.712751,-79.390197


In [44]:
loc_data = df1.reset_index(drop=True) 
loc_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.692657,-79.264848
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.778517,-79.346556
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.725900,-79.340923
3,M1G,Scarborough,Woburn,43.695344,-79.318389
4,M1H,Scarborough,Cedarbrae,43.712751,-79.390197
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,NaN,NaN
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",NaN,NaN
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",NaN,NaN
